In [ ]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# AIDO Tutorial

The aim of this tutorial is to show (on a simple example) the 7 main steps of the AIDO workflow. 

**Table of Contents**


## 1-Specify 

This tutorial focuses on the design and optimization of an **extruded aluminum heat sink** for mid-power electronic devices (15W–150W), such as LED lighting, CPUs, or telecom equipment. Poor thermal management can lead to overheating, reduced performance, and shorter product lifespans — making heat sink design a key differentiator in modern electronics.

The chosen geometry is inspired by real products (e.g., HeatSinkUSA 10.080 https://www.heatsinkusa.com/10-080-wide-extruded-aluminum-heatsink/), widely used for their efficiency, low cost, and lightweight profile.


![Extruded Aluminum Heatsink](heatsink.jpg)

## 2-Model

In this section, we translate our design specification into a computational model. This model combines **geometric calculations**, **material properties**, and **fluid/thermal physics** to simulate the heat sink’s behavior under operating conditions.

#### 📐 Default Design Space
- **Base**: 100 × 50 mm  
- **Fin Height**: 25 mm  
- **Fin Thickness**: 1.5 mm  
- **Fin Spacing**: 5 mm  
- **Number of Fins**: ~10  
- **Surface Area**: ~500 cm²  
- **Volume**: ~0.0075 m³  
- **Mass**: ~202 g  
- **Material** : Aluminum 6063-T5*  
- **Thermal Conductivity**: 201 W/m·K  
- **Corrosion Resistance**: Excellent  


#### Outpouts 
- 🔻 Thermal Resistance
- 🔼 Fin Efficiency    
- ⚖️ Total Mass        
- 💨 Pressure Drop     





# 3- Calibrate

**Calibration** is the process of adjusting model parameters so that its outputs match real-world data. This step ensures that our simulations reflect physical behavior as closely as possible.

In this tutorial, we will use experimental data provided in the file: 📄 `experimental_data.csv`

To calibrate the model, we’ll compare its outputs (e.g., max temperature, pressure drop) to the experimental values, and minimize the **mean squared error (MSE)** between them. The calibration is done using a simple optimization routine provided in the code cells below.




Let's now calibrate the model using our experimetal data

# 4- Compute

In this step, we run **batch simulations** to explore the design space using a **Latin Hypercube Sampling (LHS)** Design of Experiments (DoE). This allows us to efficiently sample a wide range of geometric and physical parameters. Each simulation returns performance metrics (thermal resistance, fin efficiency, mass, pressure drop), which will be used in the next steps for AI reduction and optimization.


🚀 In many use-cases, simulations can be **computationally intensive**, they are then offloaded to the cloud and executed on **High Performance Computing (HPC)** infrastructure (e.g. using **Rescale API**  )

# 5- Reduce (AI)

With the batch simulation data in hand, we now train **surrogate models** to approximate the system behavior. These models act as lightweight predictors of performance metrics, enabling fast evaluations during optimization.



This step is essential because:

✅ It drastically reduces computation time  
✅ It enables real-time and interactive exploration  
✅ It enables design optimization (inverse problem)


# 6- Optimize


This final step is the **core purpose of the entire workflow**: finding the best designs based on defined performance objectives.

Using the **surrogate models** trained earlier, we can now run fast and efficient **optimization loops** — without re-running costly simulations.

The full potential lies in the **multi-objective optimization**, where trade-offs between thermal performance, weight, pressure drop, and cost can be explored 
But, in this tutorial, we demonstrate a **single-objective optimization** for simplicity : 

For example we want to achieve values :
-  `Target` Thermal Resistance $T_{\text{target}}$ 
-  `Target` Pressure Drop  $P_{\text{target}}$ 
using the single objective 

$$
f = (T_{\text{pred}} - T_{\text{target}})^2 + (P_{\text{pred}} - P_{\text{target}})^2
$$


You’ve now completed a full AIDO workflow!

In [ ]:
import numpy as np
x_opt = opt.optimize(method="DE", n_var=3,verbose=True, n_evals=200)
y_opt, _, _ = reduced_model.predict(x_opt.reshape(1, -1), alpha=0.05)
f_opt = my_objective_function(y_opt, opt.target_values)
print("===== DE =====")
print(f"{x_opt =}")
print(f"{y_opt =}")
print(f"{f_opt =}")

print("Optimal design parameters :")
for var, val in zip(list_variables, x_opt):
    print(f"  {var}: {val:.6f}")
print("\nTarget values:")
print(f"  Thermal Resistance Target: {thermal_resistance_target}")
print(f"  Pressure Drop Target: {pressure_drop_target}")

In [ ]:
import ns_aido.utils as utls
import ns_aido.MLReducedModel as ML
import sklearn.metrics as skmetrics

X_train, X_test, y_train, y_test = utls.train_test_split(X, y, test_size=0.1)
reduced_model = ML.MLReducedModel(  list_variables=list_variables,
    list_targets=list_targets,X=X, y=y, method="gradient_boosting_cv+")
reduced_model.fit(X_train=X_train, y_train=y_train)
y_pred, y_pred_lower, y_pred_upper = reduced_model.predict(X_test, alpha=0.05)
utls.prediction_vs_true_plot(y_test, y_pred, y_pred_lower, y_pred_upper, list_targets)

In [ ]:
# Visualize current model performance 
params_dict = ex1_params_dict.copy()
simulated_results = {"heat_load": [], "max_temperature": []}
for i in range(len(data_df)):
        params_dict["heat_load"] = data_df["heat_load"][i]
        res = model_heatsink(params_dict)
        simulated_temp = res["thermal_performance"]["maximum_temperature"]
        simulated_results["heat_load"].append(data_df["heat_load"][i])
        simulated_results["max_temperature"].append(simulated_temp)

# plot results 
plt.scatter(simulated_results["heat_load"], simulated_results["max_temperature"], color='red', marker='x')
plt.grid(True)
plt.xlabel("Heat Load (W)", color='darkblue', fontsize=12, fontweight='bold')
plt.ylabel("Max Temperature (K)", color='darkblue', fontsize=12, fontweight='bold')
plt.title("Simulated Data: Heat Load vs Max Temperature", color='darkred', fontsize=14, fontweight='bold')

plt.show()


In [ ]:
# Plot both experimental and simulated data for comparison
plt.scatter(data_df["heat_load"], data_df["max_temperature"], color='blue', label='Experimental Data')
plt.scatter(simulated_results["heat_load"], simulated_results["max_temperature"], color='red', marker='x', label='Simulated Data')
plt.grid(True)
plt.xlabel("Heat Load (W)", color='darkblue', fontsize=12, fontweight='bold')
plt.ylabel("Max Temperature (K)", color='darkblue', fontsize=12, fontweight='bold')
plt.title("Comparison of Experimental and Simulated Data", color='darkred', fontsize=14, fontweight='bold')
plt.legend()
plt.show()

In [ ]:
from scipy.stats import qmc
def lhs(n, samples):
    sampler = qmc.LatinHypercube(d=n)
    sample = sampler.random(n=samples)
    return sample

def run_batch_simu( input_df, black_box_model):
    print("Running batch simulation")

    def get_black_box_results(row):
        params_dict = row.to_dict()
        result = black_box_model(params_dict)
        return pd.Series(result)

    # Apply the function to each row in the dataframe
    results_df = input_df.apply(get_black_box_results, axis=1)
    # Concatenate the original dataframe with the results
    final_df = pd.concat([input_df, results_df], axis=1)
    df_path = "..\\results\\batch_results.csv"
    final_df.to_csv(df_path, index=False)
    print("Batch simulation completed")
    print("Data saved in {}".format(df_path))
    print("dataFrame shape :", final_df.shape)

    return df_path


# Define the ranges for the geometry parameters
param_ranges = {
    "base_length": (0.05, 0.15),  # m
    "base_width": (0.03, 0.1),  # m
    "base_thickness": (0.002, 0.01),  # m
    "fin_height": (0.01, 0.05),  # m
    "fin_thickness": (0.001, 0.005),  # m
    "fin_spacing": (0.002, 0.01),  # m
}

# Generate 1000 Latin HyperCube samples
lhs_samples = lhs(len(param_ranges), samples=1000)
scaled_samples = []
for i, (param, (low, high)) in enumerate(param_ranges.items()):
    scaled_samples.append(lhs_samples[:, i] * (high - low) + low)
input_df = pd.DataFrame(
    {param: scaled_samples[i] for i, param in enumerate(param_ranges.keys())}
)

# Add the constant parameters to the dataframe
for key, value in params_dict.items():
    if key not in input_df.columns:
        input_df[key] = value


# Submit the batch simulation job 
output_path = run_batch_simu(input_df, black_box_model)

In [ ]:
from ns_aido.heatsink import visualize_heatsink_geometry, black_box_model, model_heatsink

ex1_params_dict = {
    "name": "Extruded Aluminum Heatsink",
    "base_length": 0.1,  # m
    "base_width": 0.05,  # m
    "base_thickness": 0.005,  # m
    "fin_height": 0.025,  # m
    "fin_thickness": 0.0015,  # m
    "fin_spacing": 0.005,  # m
    "thermal_conductivity": 10,  # W/(m·K)
    "specific_heat_capacity": 900,  # J/kg·K
    "weight": 0.202,  # kg
    "heat_load": 100,  # W
    "ambient_temp": 298,  # K (25°C)
    "air_velocity": 2,  # m/s
}
visualize_heatsink_geometry(ex1_params_dict)
output = black_box_model(ex1_params_dict)

# Results before any calibration
results = model_heatsink(ex1_params_dict)
results['thermal_performance']['maximum_temperature']

In [ ]:
import numpy as np
from ns_aido.SingleObjectiveOptimizer_rm import SingleObjectiveOptimizer

thermal_resistance_target = 0.08135199
pressure_drop_target  = 1.66261048
# These targets are achieved with the following parameters (in a real case, we would not know them of course)
X_targ = [0.11786249, 0.06709637, 0.00511829, 0.03261981, 0.0024487 ,0.00256737]


# create an optimizer object with its objective function and its surrogate model
opt = SingleObjectiveOptimizer()
opt.forward_model = reduced_model
opt.target_values = np.array([thermal_resistance_target, pressure_drop_target])
def my_objective_function(y_pred, target_values):
    return np.sum((y_pred - target_values) ** 2)
opt.objective_function = my_objective_function

# set an initial guess and the bounds
opt.init_guess = np.mean(X, 0)
opt.xl = loader.lower_bounds
opt.xu = loader.upper_bounds

In [ ]:
from scipy.optimize import minimize

def calib_objective_function(params_to_calibrate, params_to_calibrate_names, params_dict, experimental_df):
    # Update the parameters to be calibrated
    for i, param_name in enumerate(params_to_calibrate_names):
        if param_name in params_dict:
            params_dict[param_name] = params_to_calibrate[i]
        else:
            raise ValueError(
                f"Parameter {param_name} not found in the parameters dictionary"
            )
    total_error = 0
    for i in range(len(experimental_df)):
        params_dict["heat_load"] = experimental_df["heat_load"][i]
        res = model_heatsink(params_dict)
        simulated_temp = res["thermal_performance"]["maximum_temperature"]
        experimental_temp = experimental_df["max_temperature"][i]
        total_error += (simulated_temp - experimental_temp) ** 2
    print(total_error)
    return total_error

def calibrate(
    params_to_calibrate_names, initial_guess, bounds, params_dict, experimental_df
):
    print("input params_dict", params_dict)
    result = minimize(
        calib_objective_function,
        initial_guess,
        args=(params_to_calibrate_names, params_dict, experimental_df),
        bounds=bounds,
        method="L-BFGS-B",
    )
    # Update the parameters with the calibrated values
    for i, param_name in enumerate(params_to_calibrate_names):
        if param_name in params_dict:
            params_dict[param_name] = result.x[i]
        else:
            raise ValueError(
                f"Parameter {param_name} not found in the parameters dictionary"
            )
    print("output params_dict", params_dict)
    return params_dict

In [ ]:
import sys
sys.path.append("../../src")
import ns_aido.DataLoader as DL


# Load the batch simulation results
output_path = "../results/batch_results.csv"
loader = DL.DataLoader()
loader.read_data(output_path)

# Define the input variables and the output targets
list_variables = [
    "base_length",
    "base_width",
    "base_thickness",
    "fin_height",
    "fin_thickness",
    "fin_spacing",
]
list_targets = ["thermal_resistance", "pressure_drop"]
loader.set_inputs(list_variables)
loader.set_outputs(list_targets)
X, y = loader.get_Xy()

print(f"{X.shape=}")
print(f"{y.shape=}")

In [ ]:
data_df = pd.read_csv("..\\data\\experimental_data.csv")
# Obtain plot of data_df 
plt.scatter(data_df["heat_load"], data_df["max_temperature"], color='blue')
# Modify layout
plt.grid(True)
plt.xlabel("Heat Load (W)", color='darkblue', fontsize=12, fontweight='bold')
plt.ylabel("Max Temperature (K)", color='darkblue', fontsize=12, fontweight='bold')
plt.title("Experimental Data: Heat Load vs Max Temperature", color='darkred', fontsize=14, fontweight='bold')

plt.show()


In [ ]:
from sklearn.metrics import mean_squared_error

data_df = pd.read_csv("..\\data\\experimental_data.csv")
params_to_calibrate_names = [
    "thermal_conductivity",
]  # Parameters to be calibrated
initial_guess = [200]
bounds = [
    (100, 400),
]  # Example bounds for thermal_conductivity and density
params_dict = calibrate(
    params_to_calibrate_names, initial_guess, bounds, ex1_params_dict, data_df
)
# Simulate again with the calibrated parameters
data_df["max_temperature_simu_calibrated"] = np.nan
for i in range(len(data_df)):
    params_dict["heat_load"] = data_df["heat_load"][i]
    res = model_heatsink(params_dict)
    data_df["max_temperature_simu_calibrated"][i] = res["thermal_performance"][
        "maximum_temperature"
    ]

# Plot the results
plt.figure(figsize=(10, 6))
plt.scatter(
    data_df["heat_load"],
    data_df["max_temperature"],
    color="blue",
    label="Experimental Data",
)
plt.scatter(
    data_df["heat_load"],
    data_df["max_temperature_simu_calibrated"],
    color="red",
    marker="x",
    label="Simulation (Calibrated)",
)

# Calculate MSE between experimental and calibrated simulation results

mse = mean_squared_error(data_df["max_temperature"], data_df["max_temperature_simu_calibrated"])

# Calculate relative error (mean absolute percentage error)
relative_error = (
    np.abs(data_df["max_temperature"] - data_df["max_temperature_simu_calibrated"])
    / np.abs(data_df["max_temperature"])
).mean() * 100

plt.text(
    0.05,
    0.95,
    f"MSE = {mse:.2f}\nRel. Error = {relative_error:.2f}%",
    transform=plt.gca().transAxes,
    fontsize=12,
    verticalalignment="top",
    bbox=dict(boxstyle="round", facecolor="white", alpha=0.7),
)


plt.title("Comparison of Experimental and Simulated Data", color='darkred', fontsize=14, fontweight='bold')
plt.xlabel("Heat Load (W)", color='darkblue', fontsize=12, fontweight='bold')
plt.ylabel("Max Temperature (K)", color='darkblue', fontsize=12, fontweight='bold')
plt.grid(True)
plt.legend()
plt.show()

